In [3]:
from fnl_pipe.util import OutputManager, get_yaml_dict, average_fl, downgrade_fl, get_fname, parse_act_beam
from fnl_pipe.cmb import ACTPipe, ACTMetadata
from fnl_pipe.gal_cmb import CMBxGalPipe
from fnl_pipe.galaxy import DESILSCat, AndCut, NullCut, LRGNorthCut, LRGSouthCut, ZerrCut

from pixell import enplot, enmap

import matplotlib.pyplot as plt
import numpy as np
from scipy.optimize import minimize

import sys

# beam: "/home/aroman/data/act_pub/act_planck_dr5.01_s08s18_f090_daynight_beam.txt"
# cl_tt_act: "/data/aroman/symmetry/desils/cl_tt_act_kendrick_data_f090.npy"
# cl_tt_sim: "/data/aroman/symmetry/desils/cl_tt_sim_kendrick_data_f090.npz"
# cl_tz_sig: "/data/aroman/symmetry/desils/cl_tz_sig_kendrick_data_vmeansub_f090.npy"
# cl_tz_mc: "/data/aroman/symmetry/desils/cl_tz_mc_kendrick_data_vmeansub_f090_nmc_1024.npy"
# map_weight: "/home/aroman/data/kendrick/01_act_ps_model_inputs/wfkp_90.fits"
# fl: "/data/aroman/symmetry/desils/fl_apparent_camb_f090.npy"

cl_tz_sig = np.load("/data/aroman/symmetry/desils/cl_tz_sig_kendrick_data_vmeansub_f090.npy")
cl_tz_mc = np.load("/data/aroman/symmetry/desils/cl_tz_mc_kendrick_data_vmeansub_f090_nmc_1024.npy")
cl_tz_std = cl_tz_mc.std(axis=0)
# cl_tz_mc_red = [downgrade(cl_tz_, nave_tz) for ]

lmax = len(cl_tz_sig) - 1

ells = np.arange(lmax + 1)

nave_tz = 500

# lmin_fit = 2000
# lmax_fit = 7000
# ells_fit = ells[lmin_fit:lmax_fit]
# cl_tz_fit = cl_tz_sig[lmin_fit:lmax_fit]
# ivar_weight = 1. / cl_tz_std[lmin_fit:lmax_fit]**2
lmin_fit = 4
lmax_fit = 14
ells_fit = downgrade_fl(ells, nave_tz)[lmin_fit:lmax_fit]
cl_tz_fit = downgrade_fl(cl_tz_sig, nave_tz)[lmin_fit:lmax_fit]
ivar_weight = 1. / downgrade_fl(cl_tz_std, nave_tz)[lmin_fit:lmax_fit]**2

coeff = 1e-4


# def _eval_neighborhood(fun, x, r):
#     xs = np.array([[1.,1.],
#                    [r, r],
#                    [1/r, r],
#                    [r ,1/r],
#                    [1/r, 1/r]]) * x[None,:]
    
#     fevals = [fun(xt) for xt in xs]
#     imin = np.argmin(fevals)
   
#     xmin = xs[imin]
#     fmin = fevals[imin]
    
#     newmin=False
#     if imin != 0: newmin=True
    
#     return newmin, xmin, fmin
    

# def logspace_min(fun, x, rel_tol, abs_tol,
#             r=0.8, niter_refine=5, iiter_refine=0,
#             last_min=None, rdecel):
    
#     if last_min is None:
#         last_min = fun(x)

#     _eval_neighborhood(fun, x, r)               
    
#     if np.abs(fmin - last_min) / last_min < 

def _bs_return_1d(fl, fc, fr, xl, xc, xr):
    imin = np.argmin(fl, fc, fr)
    return [xl, xc, xr][imin], [fl, fc, fr][imin]


def _bs_1d(fun, xl, xr, fmin, rtol): 
    fl = fun(xl)
    fc = fun(0.5*(xl + xr))
    fr = fun(xr)
    
    assert fc <= fmin
    
    fminmin = min(fl, fc, fr)
    
    if np.abs(fmin - fc) / fmin <= rtol:
        return _bs_return_1d(fl, fc, fr, xl, xc, xr)
                  
    if fl < fr:
        return _bs_1d(fun, xl, xc, fc, rtol)
    else:
        return _bs_1d(fun, xc, xr, fc, rtol)

def _bs_1d_1iter(fun, xl, xr, xmin, fl, fr, fmin):
    xc = 0.5*(xl + xr)
    fc = fun(xc)
    
    assert fc <= fmin
    
    if fl <= fr:
        return xl, xc, xc, fl, fc, fc
    return xc, xr, xc, fc, fr, fc

class Bs2dState:
    def __init__(self, x)

def bs_2d(fun, xlim, ylim, fmin, xstate, ystate, niter_left=20, rtol=0.001):
    
    xfun = lambda x: fun([x, ylim[0]])
    fxl = xfun(xlim[0])
    fxr = xfun(xlim[1])

    xl, xr, xmin, fxl, fxr, fmin = _bs_1d_1iter(xfun, xlim[0], xlim[1], )

    xmin = [xl, xr][np.argmin(fxl, fxr)]
    yfun = lambda y: fun([xmin, y])
    yl, yr, fyl, fyr = _bs_1d_1iter(yfun, ylim[0], ylim[1], yfun(ylim[0]), yfun([ylim[1]]))

    
def exp_model_fit(x):
    return x[0] * np.exp(-(ells_fit/x[1])**2)

def opt_fun(x):
    res = ((coeff * (exp_model_fit(x) - cl_tz_fit)**2) * ivar_weight).sum()
    return res

a0 = cl_tz_sig[2000]
# x0 =[a0, 6000.]
x0 = [1., 8000.]
res = minimize(opt_fun, x0)

l0_eval = np.linspace(7000., 8000., 128)
x_eval = [[-7.45058393e-09,  l0] for l0 in l0_eval]
feval = [opt_fun(x) for x in x_eval]

plt.figure(dpi=300, facecolor='w')
plt.plot(feval)


print('===================== OPT RESULT =====================')
print(res)
print('=================== END OPT RESULT ===================')

SyntaxError: invalid syntax (4218714716.py, line 105)